In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from twitter.twitter_collection import TwitterCollection
from twitter.tweet_sentiment_analyzer import TweetSentimentAnalyzer

import yfinance as yf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action = 'ignore', category = pd.errors.PerformanceWarning)
%matplotlib inline

Python-dotenv could not parse statement starting at line 11
[nltk_data] Downloading package stopwords to /home/kozan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
client_twitter = TwitterCollection()  
tsa = TweetSentimentAnalyzer() 

Authentication Successfull


In [21]:
def create_sentiment_scores(df_tweets: pd.DataFrame()) -> pd.DataFrame():
    df = tsa.cleaning_tweet_data(df_tweets)
    df = tsa.preprocessing_tweet_datetime(df)
    df = tsa.get_sentiment_scores(df)
    tsa.add_datetime_to_col(df)
    return df  

i = 'ada'
df_tweets = client_twitter.get_tweets_with_interval(i, 'en', hour=24*3, interval=4)
print(f'shape of {i} tweets df: {df_tweets.shape}')
path_df = f'../data/tweets_data/{i}/'
file_df = f'{i}_tweets.csv'
#client_twitter.write_tweets_csv(df_tweets, path_df, file_df)

sent_day_file_df = f'{i}_day.csv'
sent_hour_file_df = f'{i}_hour.csv'

if not os.path.exists(os.path.join(path_df, sent_day_file_df)):
    os.makedirs(path_df, exist_ok=True)
    with open(os.path.join(path_df, sent_day_file_df), mode='a'): pass
    df = create_sentiment_scores(df_tweets)

    df_result_day = tsa.get_sent_with_mean_interval(df, interval='1d')
    #df_result_day.to_csv(os.path.join(path_df, sent_day_file_df))

    df_result_hour = tsa.get_sent_with_mean_interval(df, interval='1h')
    #df_result_hour.to_csv(os.path.join(path_df, sent_hour_file_df))
else:
    sent_day_df = pd.read_csv(os.path.join(path_df, sent_day_file_df), index_col=[0])
    sent_day_df.index = pd.to_datetime(sent_day_df.index)
    sent_hour_df = pd.read_csv(os.path.join(path_df, sent_hour_file_df), index_col=[0])
    sent_hour_df.index = pd.to_datetime(sent_hour_df.index)
    print(f'day sentiment:\n {sent_day_df}')
    df = create_sentiment_scores(df_tweets)
    df_result_day = tsa.get_sent_with_mean_interval(df, interval='1d')
    df_result_day.index = pd.to_datetime(df_result_day.index)
    #df_result_day.to_csv(os.path.join(path_df, sent_day_file_df))
    print(f'day results:\n {df_result_day}')
    sent_tweets = pd.concat([sent_day_df, df_result_day[df_result_day.index>sent_day_df.index[-1]]])
    print(f'day total:\n {sent_tweets}')
    df_result_hour = tsa.get_sent_with_mean_interval(df, interval='1h')
    #client_twitter.write_tweets_csv(df_tweets, path_df, file_df)

shape of ada tweets df: (55495, 8)
day sentiment:
             compound_total
Date                      
2022-12-04        0.416257
2022-12-05        0.301695
2022-12-06        0.354274
2022-12-07        0.366592
2022-12-08        0.396052
2022-12-09        0.222896
2022-12-10        0.412876
2022-12-11        0.520940
day results:
             compound_total
Date                      
2022-12-10        0.185480
2022-12-11        0.537989
2022-12-12        0.335477
2022-12-13        0.132820
day total:
             compound_total
Date                      
2022-12-04        0.416257
2022-12-05        0.301695
2022-12-06        0.354274
2022-12-07        0.366592
2022-12-08        0.396052
2022-12-09        0.222896
2022-12-10        0.412876
2022-12-11        0.520940
2022-12-12        0.335477
2022-12-13        0.132820


In [17]:
df_result_day.index = pd.to_datetime(df_result_day.index)
type(df_result_day.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [18]:
df_result_day.index

DatetimeIndex(['2022-12-10', '2022-12-11', '2022-12-12', '2022-12-13'], dtype='datetime64[ns]', name='Date', freq=None)

In [19]:
sent_day_df.index

Index(['2022-12-04', '2022-12-05', '2022-12-06', '2022-12-07', '2022-12-08',
       '2022-12-09', '2022-12-10', '2022-12-11'],
      dtype='object', name='Date')